In [1]:
%load_ext autotime

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["SUFFIX"]= "1500K-with-3-vids"
os.environ["BASE_DIR"] = "/home/ec2-user/emb3"

hyperparameters = {
    'loss': "adaptive_hinge",
    'batch_size': 256,
    'learning_rate': 1e-3,
    'l2': 1e-06,
    'n_iter': 30,
    'embedding_dim': 64
}

import pandas as pd
import numpy as np
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.interactions import Interactions
import pandas as pd
from spotlight.evaluation import *
from torch.utils.tensorboard import SummaryWriter
import torch
from torch.utils.tensorboard import SummaryWriter
import logging
from spotlight.factorization.representations import *
import os
import collections

from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.sequence.representations import CNNNet

In [4]:
model_name = "cnn_1500K-with-3-vids_loss=adaptive_hinge,batch=128,lr=0.001,l2=1e-06,n_iter=50,emb_dim=128,type=cnn,kernel_width=3,dilation=1,num_layers=2,nonlinearity=tanh,residual=True,mom=0.9,neg=3,amsgrad=True,adamw=False"
base_model_dir="/home/ec2-user/emb3/smodels"
epoch=38
mm = torch.load("{}/{}-{}".format(base_model_dir,model_name,epoch))

time: 3.09 s


In [38]:
embs = mm.get_embeddings()
suffix = os.environ["SUFFIX"]
aa = pd.read_parquet(os.environ['BASE_DIR'] + "/data/video2index-pandas-aug-28-phase" + suffix)
videoid2index = dict(zip(aa["k"], aa["v"]))
index2videoid = dict(zip(videoid2index.values(), videoid2index.keys()))

time: 12.6 ms


In [39]:
video_embs_list = []
for i, em in enumerate(embs.cpu().detach().numpy()):
    video_embs_list.append((index2videoid[i], i, em))

time: 15.4 ms


In [45]:
pd.DataFrame(video_embs_list, columns=["video_id_int", "vindex", "vector"]).to_parquet("/home/ec2-user/emb3/data/layer-2-embs-epoch-" + str(epoch))

time: 136 ms


In [41]:
als_embds = pd.read_parquet("/home/ec2-user/emb3/data/als-embs-pandas-aug-28-phase" + suffix)

time: 65.6 ms


In [42]:
als_embds["video_id_int"] = als_embds["video_id"]
als_embds[["video_id_int", "vindex", "vector"]].to_parquet("/home/ec2-user/emb3/data/als-embeds")

time: 166 ms


In [43]:
aa.to_parquet(os.environ['BASE_DIR'] + "/data/video2index-pandas-aug-28-for-embeds")

time: 7.81 ms
